<a href="https://colab.research.google.com/github/zameer-Gimba/Recommendation_System/blob/main/DL_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-13 15:44:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  79.6MB/s    in 0.3s    

2025-03-13 15:44:31 (79.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Load datasets
books = pd.read_csv("BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines="skip", low_memory=False)
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding="latin-1", on_bad_lines="skip", low_memory=False)
users = pd.read_csv("BX-Users.csv", sep=";", encoding="latin-1", on_bad_lines="skip", low_memory=False)

In [ ]:
# Rename columns for easier access
books.columns = ["ISBN", "Title", "Author", "Year", "Publisher", "Image1", "Image2", "Image3"]
ratings.columns = ["UserID", "ISBN", "Rating"]
users.columns = ["UserID", "Location", "Age"]

In [ ]:
# Filter out users who rated less than 200 books
user_counts = ratings["UserID"].value_counts()
active_users = user_counts[user_counts >= 200].index
ratings = ratings[ratings["UserID"].isin(active_users)]

In [ ]:
# Filter out books that were rated less than 100 times
book_counts = ratings["ISBN"].value_counts()
popular_books = book_counts[book_counts >= 100].index
ratings = ratings[ratings["ISBN"].isin(popular_books)]

In [ ]:
# Merge ratings with book titles
book_ratings = ratings.merge(books[["ISBN", "Title"]], on="ISBN")

In [ ]:
# Create user-book matrix
user_book_matrix = book_ratings.pivot_table(index="Title", columns="UserID", values="Rating").fillna(0)

In [ ]:
# Convert to sparse matrix
book_sparse_matrix = csr_matrix(user_book_matrix)

In [ ]:
# Train KNN Model
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(book_sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# Function to get book recommendations
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864549790632], ["The Pilot's Wife : A Novel", 0.8192678336213406], ['The Joy Luck Club', 0.8198604785739199], ['The Notebook', 0.8236682900571164], ['Bel Canto: A Novel', 0.8247874880681116]]]
You haven't passed yet. Keep trying!
